In [1]:
#This script prepare the data fro LMD annotation. The data has to be copied into the annotation sheets
import json
import os
import math
import re
import numpy as np

types = {"Island": "ISL", "State": "STAT", "Continent": "CONT", "City/town": "CITY", "Country": "CTRY",
        "County": "CNTY", "Neighborhood": "NBHD", "Road/street": "ST", "District": "DIST", "Other locations": "OTHR", 
        "Natural Point-of-Interest": "NPOI", "Human-made Point-of-Interest": "HPOI"}


In [2]:
# reads the BILOU formatted data

def read_bilou(path):
    tokens = []
    labels = []
    t = []
    l = []
    
    for token in open(path, encoding='utf-8').read().splitlines(): 
        if token == '':
            tokens.append(t)
            labels.append(l)
            t = []
            l = []
            continue
        splits = token.split()
        t.append(splits[0])
        l.append(splits[1])
        
    if len(t) > 0 and len(l) > 0:
        tokens.append(t)
        labels.append(l) 
        
    return tokens, labels

In [3]:
def remove_punct(text):
    punct = re.compile(r'(\w+)')
    clean_text = ' '.join([m.group() for m in punct.finditer(text)])
    return clean_text

In [4]:
def parse_json(path):
    locs = []
    lines = open(path, encoding='utf-8').read().splitlines()
    for line in lines:
        tweet = json.loads(line)
        locs.append([remove_punct(lm["text"]).lower().replace(" ","") for lm in tweet["location_mentions"]])
    return locs

def parse_json_by_type(path, LOC):
    locs = []
    lines = open(path, encoding='utf-8').read().splitlines()
    for line in lines:
        tweet = json.loads(line)
        if LOC:
            locs.append([(remove_punct(lm["text"]).lower().replace(" ",""), "LOC") for lm in tweet["location_mentions"]])
        else:
            locs.append([(remove_punct(lm["text"]).lower().replace(" ",""), types[lm["type"]]) for lm in tweet["location_mentions"]])
    return locs

In [5]:
def parse_bilou(path):
    tokens, labels = read_bilou(path)
        
    locs = []

    for i in range(len(tokens)):
        loc = ""
        local_locs = []
        for j in range(len(tokens[i])):
            if "U-" in labels[i][j]:
                loc = tokens[i][j]#.replace(" ’", "’").replace(" - ", "-").replace("# ", "#").replace(" ,", ",").replace(" / ", "/")
                local_locs.append(remove_punct(loc).lower().replace(" ",""))
                loc = ""
            elif "B-" in labels[i][j]: #malformed BIO-LM will be ignored
                loc = tokens[i][j]
            elif "I-" in labels[i][j]:
                if loc == "": #malformed BIO-LM will be ignored
                    continue 
                loc += " " + tokens[i][j]
            elif "L-" in labels[i][j]:
                if loc == "": #malformed BIO-LM will be ignored
                    continue 
                loc += " " + tokens[i][j]
                #local_locs.append(loc.replace(" ’", "’").replace(" - ", "-").replace("# ", "#").replace(" ,", ",").replace(" / ", "/"))
                local_locs.append(remove_punct(loc).lower().replace(" ",""))
                loc = ""
            else:
                loc = ""  #malformed BIO-LM will be ignored
        locs.append(local_locs)
    return locs

def parse_bilou_by_type(path):
    tokens, labels = read_bilou(path)
    
    locs = []

    for i in range(len(tokens)):
        loc = ""
        local_locs = []
        for j in range(len(tokens[i])):
            if "U-" in labels[i][j]:
                loc = tokens[i][j]
                local_locs.append((remove_punct(loc).lower().replace(" ",""), labels[i][j].split("-")[1]))
                loc = ""
            elif "B-" in labels[i][j]: #malformed BIO-LM will be ignored
                loc = tokens[i][j]
            elif "I-" in labels[i][j]:
                if loc == "": #malformed BIO-LM will be ignored
                    continue 
                loc += " " + tokens[i][j]
            elif "L-" in labels[i][j]:
                if loc == "": #malformed BIO-LM will be ignored
                    continue
                loc += " " + tokens[i][j]
                local_locs.append((remove_punct(loc).lower().replace(" ",""), labels[i][j].split("-")[1]))
                loc = ""
            else:
                loc = ""  #malformed BIO-LM will be ignored
                
        locs.append(local_locs)
    return locs

In [6]:
def parse_bio(path):
    tokens, labels = read_bilou(path)
        
    locs = []

    for i in range(len(tokens)):
        loc = ""
        local_locs = []
        for j in range(len(tokens[i])):
            if labels[i][j] != "O":
                try:
                    if labels[i][j].split("-")[1] in ['PERS', 'ORG']:
                        continue
                except:
                    print(labels[i][j])
            
            if "B-" in labels[i][j]: #malformed BIO-LM will be ignored
                if loc != "":
                    local_locs.append(remove_punct(loc).lower().replace(" ",""))
                loc = tokens[i][j]
            elif "I-" in labels[i][j]:
                if loc == "": #malformed BIO-LM will be ignored
                    continue 
                loc += " " + tokens[i][j]
            else:
                if loc != "":
                    local_locs.append(remove_punct(loc).lower().replace(" ",""))
                loc = ""  #malformed BIO-LM will be ignored
        locs.append(local_locs)
    return locs

def parse_bio_by_type(path):
    tokens, labels = read_bilou(path)
    
    locs = []

    for i in range(len(tokens)):
        loc = ""
        local_locs = []
        for j in range(len(tokens[i])):
            if "B-" in labels[i][j]: #malformed BIO-LM will be ignored
                if loc != "":
                    local_locs.append((remove_punct(loc).lower().replace(" ",""), labels[i][j].split("-")[1]))
                loc = tokens[i][j]
            elif "I-" in labels[i][j]:
                if loc == "": #malformed BIO-LM will be ignored
                    continue 
                loc += " " + tokens[i][j]
            else:
                if loc != "":
                    local_locs.append((remove_punct(loc).lower().replace(" ",""), labels[i][j].split("-")[1]))
                loc = ""  #malformed BIO-LM will be ignored
        locs.append(local_locs)
    return locs

In [7]:
def parse_bilou(path):
    tokens, labels = read_bilou(path)
        
    locs = []

    for i in range(len(tokens)):
        loc = ""
        local_locs = []
        for j in range(len(tokens[i])):
            if "U-" in labels[i][j]:
                loc = tokens[i][j]#.replace(" ’", "’").replace(" - ", "-").replace("# ", "#").replace(" ,", ",").replace(" / ", "/")
                local_locs.append(remove_punct(loc).lower().replace(" ",""))
                loc = ""
            elif "B-" in labels[i][j]: #malformed BIO-LM will be ignored
                loc = tokens[i][j]
            elif "I-" in labels[i][j]:
                if loc == "": #malformed BIO-LM will be ignored
                    continue 
                loc += " " + tokens[i][j]
            elif "L-" in labels[i][j]:
                if loc == "": #malformed BIO-LM will be ignored
                    continue 
                loc += " " + tokens[i][j]
                #local_locs.append(loc.replace(" ’", "’").replace(" - ", "-").replace("# ", "#").replace(" ,", ",").replace(" / ", "/"))
                local_locs.append(remove_punct(loc).lower().replace(" ",""))
                loc = ""
            else:
                loc = ""  #malformed BIO-LM will be ignored
        locs.append(local_locs)
    return locs

def parse_bilou_by_type(path):
    tokens, labels = read_bilou(path)
    
    locs = []

    for i in range(len(tokens)):
        loc = ""
        local_locs = []
        for j in range(len(tokens[i])):
            if "U-" in labels[i][j]:
                loc = tokens[i][j]
                local_locs.append((remove_punct(loc).lower().replace(" ",""), labels[i][j].split("-")[1]))
                loc = ""
            elif "B-" in labels[i][j]: #malformed BIO-LM will be ignored
                loc = tokens[i][j]
            elif "I-" in labels[i][j]:
                if loc == "": #malformed BIO-LM will be ignored
                    continue 
                loc += " " + tokens[i][j]
            elif "L-" in labels[i][j]:
                if loc == "": #malformed BIO-LM will be ignored
                    continue
                loc += " " + tokens[i][j]
                local_locs.append((remove_punct(loc).lower().replace(" ",""), labels[i][j].split("-")[1]))
                loc = ""
            else:
                loc = ""  #malformed BIO-LM will be ignored
                
        locs.append(local_locs)
    return locs

In [8]:
def pad_unpredicted_seq(gl, pl):
    if len(gl) != len(pl):
        raise NameError("Found input variables with inconsistent numbers of samples! len(gold) = %d and len(pred) = %d" % (len(gl), len(pl)))

    for i in range(len(gl)):
        if len(pl[i]) < len(gl[i]):
            rem = len(gl[i]) - len(pl[i])
            for r in range(rem):
                pl[i].append("O")
    return pl

In [9]:
def extract_matrix_per_example(gold, pred):
    TPs = []
    FPs = []
    FNs = []
    
    for i in range(len(gold)):
        ''' FIXME find more efficient and correct'''
        
        # in some cases the prediction contains same LM twice while it appear once in gold.
        TP = []
        FP = []
        #TN = []
        FN = []
        for l in pred[i]:
            if l in gold[i]:
                if gold[i].count(l) == TP.count(l):
                    FP.append(l)
                    continue
                else: #l not in TP yet
                    TP.append(l)
            else:
                FP.append(l)
                
        TPs.append(TP)
        FPs.append(FP)
        
        for l in gold[i]:
            if l not in pred[i]:
                if pred[i].count(l) == FN.count(l):
                    #TN.append(l)
                    continue
                else: #l not in FN yet
                    FN.append(l)
            #else:
            #    TN.append(l)
                
        FNs.append(FN)
        
        
    #print(TPs)
    #print(FPs)
    #print(FNs)
    
        
    return TPs, FPs, FNs

def count_matrix_per_example(gold, pred):
    
    TPs, FPs, FNs = extract_matrix_per_example(gold, pred)
    TPs_counts = [len(tp) for tp in TPs]
    FPs_counts = [len(fp) for fp in FPs]
    FNs_counts = [len(fn) for fn in FNs]

    return TPs_counts, FPs_counts, FNs_counts

In [10]:
def compute_precision(gold, pred, mode, internal=False):
    
    if len(gold) != len(pred):
        raise NameError("Found input variables with inconsistent numbers of samples! len(gold) = %d and len(pred) = %d" 
                        % (len(gold), len(pred)))
    
    # FIXME count this ONCE for all eval measures!
    TPs, FPs, FNs = count_matrix_per_example(gold, pred)
    ps = []
    if mode == 'special':
        for i in range(len(TPs)):
            #special case where no LMs in the gold and the system outputs nothing
            #it's commented because we're evaluting for the positive label (LOC) here
            if list(set(gold[i] + pred[i])) == []:
                #print("special case")
                p = 1.00
            else:
                den = len(pred[i]) # FIXME can be replaced with TPs[i] + FPs[i]
                p = TPs[i]/den if den > 0 else 0
            
            ps.append(p)
    else:    
        for i in range(len(TPs)):
            den = len(pred[i])  # FIXME can be replaced with TPs[i] + FPs[i]
            p = TPs[i]/den if den > 0 else 0 
            ps.append(p)
    #print(ps)
    if internal:
        return ps
    else:
        return sum(ps)/len(ps)

def compute_recall(gold, pred, mode, internal=False):
    if len(gold) != len(pred):
        raise NameError("Found input variables with inconsistent numbers of samples! len(gold) = %d and len(pred) = %d" 
                        % (len(gold), len(pred)))
    
    TPs, FPs, FNs = count_matrix_per_example(gold, pred)
    #print([TPs, FPs, FNs])
    
    rs = []
    if mode == 'special':
        #special case where no LMs in the gold and the system outputs nothing
        #it's commented because we're evaluting for the positive label (LOC) here
        for i in range(len(TPs)):
            if list(set(gold[i] + pred[i])) == []:
                #print("special case")
                r = 1.00
            else:        
                den = len(gold[i])  # FIXME can be replaced with TPs[i] + FNs[i]
                r = TPs[i]/den if den > 0 else 0
            rs.append(r)
    else:
        for i in range(len(TPs)):      
            den =  len(gold[i]) # FIXME can be replaced with  TPs[i] + FNs[i]
            r = TPs[i]/den if den > 0 else 0
            rs.append(r)
    #print(rs)
    if internal:
        return rs
    else:
        return sum(rs)/len(rs)

def compute_fscore(beta, gold, pred, mode):
    ps = compute_precision(gold, pred, mode, True)
    rs = compute_recall(gold, pred, mode, True)

    fs = []
    for i in range(len(ps)):
        if ps[i] + rs[i] == 0.00:
            f = 0.00
        else:
            beta2 = beta**2
            f = ((1 + beta2) * ps[i] * rs[i]) / (beta2 * ps[i] + rs[i])
        fs.append(f)
        #print("f = %f" % f)
    return sum(fs)/len(fs)


def count_special(gold, pred):
    count = 0
    for i in range(len(gold)):
        if list(set(gold[i] + pred[i])) == []:
            count += 1
    return count

## Running LOC evaluation

In [11]:
def evaluate(gold_path, gold_file_format, pred_path, pred_file_format, beta, mode, per_dataset):
    #DRIVER code

    # TODO 1. read the gold data 
    #         --> parameters: path, data format (BILOU, JSON)
    #         --> output gold: [[locs]], pred: [[locs]]
    if gold_file_format == "bilou":
        gold = parse_bilou(gold_path)
    elif gold_file_format == "bio":
        gold = parse_bio(gold_path)
    else:
        gold = parse_json(gold_path)
    
    if pred_file_format == "bilou":
        pred = parse_bilou(pred_path)
    elif pred_file_format == "bio":
        pred = parse_bio(pred_path)
    else:
        pred = parse_json(pred_path)
 
    
    #print(gold)
    #print(pred)
        
    #print(len(gold))
    #print(len(pred))
    
    #print()
    if per_dataset:
        gold = [[item for sublist in gold for item in sublist]]
        pred = [[item for sublist in pred for item in sublist]]
    
    # TODO 2. count the TP, TN, FP, FN
    #         --> parameters: average (micro, macro), gold: <tid:[locs]>, pred: <tid:[locs]>
    #         --> TP, TN, FP, FN
    # TODO 3. compute P, R, and F_b --> parameters: 
    #         --> TP, TN, FP, FN
    #         --> P, R, F_b
    p = compute_precision(gold, pred, mode)
    r = compute_recall(gold, pred, mode)
    f = compute_fscore(beta, gold, pred, mode)

    return p, r, f



In [112]:
from seqeval.metrics import precision_score, recall_score, f1_score
from seqeval.scheme import BILOU

path = "C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\data\\IDRISI\\en\\lmr\\"
events = ["california_wildfires_2018", "canada_wildfires_2016", "cyclone_idai_2019", "ecuador_earthquake_2016", 
          "greece_wildfires_2018", "hurricane_dorian_2019", "hurricane_florence_2018", "hurricane_harvey_2017", 
          "hurricane_irma_2017", "hurricane_maria_2017", "hurricane_matthew_2016", "italy_earthquake_aug_2016", 
          "kaikoura_earthquake_2016", "kerala_floods_2018", "maryland_floods_2018", "midwestern_us_floods_2019", 
          "pakistan_earthquake_2019", "puebla_mexico_earthquake_2017", "srilanka_floods_2017"]
per_dataset = False
#events = ["test"]
for case in ["random", "timebased"]:
    for event in events:
        pp = path + "baselines\\" + case + "\\CRF-LOC-cleaned-ordered\\" + event + ".txt"
        #gp = path + "annotations\\partitions\\" + case + "\\JSON\\" + event + "\\test.txt"            
        gp = path + "annotations\\partitions\\" + case + "\\BILOU-LOC-cleaned\\" + event + "\\test.txt"            
                
        pm, rm, fm = evaluate(gp, "bilou", pp, "bilou", 1, 'special', per_dataset) #standard
        print("%s\t%s\t%f\t%f\t%f" % (event, case, pm, rm, fm))

        #gt, gl = read_bilou(gp)
        #pt, pl = read_bilou(pp)
        #pl = pad_unpredicted_seq(gl, pl)
        #ps = precision_score(gl, pl, mode='strict', scheme=BILOU)
        #rs = recall_score(gl, pl, mode='strict', scheme=BILOU)
        #fs = f1_score(gl, pl, mode='strict', scheme=BILOU)
        #print("%f\t%f" % (fm, fs))



california_wildfires_2018	random	0.908846	0.887546	0.889573
canada_wildfires_2016	random	0.743590	0.745833	0.734963
cyclone_idai_2019	random	0.926923	0.881151	0.894011
ecuador_earthquake_2016	random	0.942222	0.912222	0.920106
greece_wildfires_2018	random	0.946161	0.926702	0.926195
hurricane_dorian_2019	random	0.859936	0.852303	0.850788
hurricane_florence_2018	random	0.766987	0.729469	0.737005
hurricane_harvey_2017	random	0.901544	0.878378	0.885714
hurricane_irma_2017	random	0.791344	0.776163	0.778424
hurricane_maria_2017	random	0.907979	0.877735	0.884299
hurricane_matthew_2016	random	0.941118	0.890120	0.904762
italy_earthquake_aug_2016	random	0.822034	0.813559	0.816384
kaikoura_earthquake_2016	random	0.875174	0.865411	0.865830
kerala_floods_2018	random	0.868590	0.827179	0.838291
maryland_floods_2018	random	0.931624	0.894231	0.900733
midwestern_us_floods_2019	random	0.959041	0.914037	0.922379
pakistan_earthquake_2019	random	0.900110	0.885375	0.882114
puebla_mexico_earthquake_2017	random

In [113]:
from seqeval.metrics import precision_score, recall_score, f1_score
from seqeval.scheme import BILOU

path = "C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\data\\IDRISI\\en\\lmr\\"
events = ["california_wildfires_2018", "canada_wildfires_2016", "cyclone_idai_2019", "ecuador_earthquake_2016", 
          "greece_wildfires_2018", "hurricane_dorian_2019", "hurricane_florence_2018", "hurricane_harvey_2017", 
          "hurricane_irma_2017", "hurricane_maria_2017", "hurricane_matthew_2016", "italy_earthquake_aug_2016", 
          "kaikoura_earthquake_2016", "kerala_floods_2018", "maryland_floods_2018", "midwestern_us_floods_2019", 
          "pakistan_earthquake_2019", "puebla_mexico_earthquake_2017", "srilanka_floods_2017"]
per_dataset = False
#events = ["test"]
for case in ["random", "timebased"]:
    for event in events:
        pp = path + "baselines\\" + case + "\\BERT-LOC\\en-target-" + case + "-BILOU-LOC-cleaned-" + event
        #gp = path + "annotations\\partitions\\" + case + "\\JSON\\" + event + "\\test.txt"            
        gp = path + "annotations\\partitions\\" + case + "\\BILOU-LOC-cleaned\\" + event + "\\test.txt"            
                
        pm, rm, fm = evaluate(gp, "bilou", pp, "bilou", 1, 'special', per_dataset) #standard
        print("%s\t%s\t%f\t%f\t%f" % (event, case, pm, rm, fm))

        #gt, gl = read_bilou(gp)
        #pt, pl = read_bilou(pp)
        #pl = pad_unpredicted_seq(gl, pl)
        #ps = precision_score(gl, pl, mode='strict', scheme=BILOU)
        #rs = recall_score(gl, pl, mode='strict', scheme=BILOU)
        #fs = f1_score(gl, pl, mode='strict', scheme=BILOU)
        #print("%f\t%f" % (fm, fs))



california_wildfires_2018	random	0.918205	0.933516	0.918767
canada_wildfires_2016	random	0.742308	0.760897	0.741667
cyclone_idai_2019	random	0.935192	0.918547	0.919803
ecuador_earthquake_2016	random	0.957407	0.951111	0.948847
greece_wildfires_2018	random	0.927400	0.930192	0.920785
hurricane_dorian_2019	random	0.869231	0.888718	0.874261
hurricane_florence_2018	random	0.799872	0.776328	0.777596
hurricane_harvey_2017	random	0.912484	0.900257	0.902188
hurricane_irma_2017	random	0.850000	0.846899	0.842583
hurricane_maria_2017	random	0.917954	0.907979	0.908494
hurricane_matthew_2016	random	0.956088	0.938224	0.938266
italy_earthquake_aug_2016	random	0.877119	0.877119	0.874294
kaikoura_earthquake_2016	random	0.918061	0.919805	0.914027
kerala_floods_2018	random	0.891731	0.896026	0.884968
maryland_floods_2018	random	0.923077	0.900641	0.901160
midwestern_us_floods_2019	random	0.944575	0.953852	0.938863
pakistan_earthquake_2019	random	0.892936	0.910375	0.889709
puebla_mexico_earthquake_2017	random

In [115]:
from seqeval.metrics import precision_score, recall_score, f1_score
from seqeval.scheme import BILOU

path = "C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\data\\IDRISI\\en\\lmr\\"
events = ["california_wildfires_2018", "canada_wildfires_2016", "cyclone_idai_2019", "ecuador_earthquake_2016", 
          "greece_wildfires_2018", "hurricane_dorian_2019", "hurricane_florence_2018", "hurricane_harvey_2017", 
          "hurricane_irma_2017", "hurricane_maria_2017", "hurricane_matthew_2016", "italy_earthquake_aug_2016", 
          "kaikoura_earthquake_2016", "kerala_floods_2018", "maryland_floods_2018", "midwestern_us_floods_2019", 
          "pakistan_earthquake_2019", "puebla_mexico_earthquake_2017", "srilanka_floods_2017"]
per_dataset = False
#events = ["test"]
for case in ["random", "timebased"]:
    for event in events:
        pp = path + "baselines\\" + case + "\\GazPNE\\" + event + ".txt"
        #gp = path + "annotations\\partitions\\" + case + "\\JSON\\" + event + "\\test.txt"            
        gp = path + "annotations\\partitions\\" + case + "\\BILOU-LOC-cleaned\\" + event + "\\test.txt"            
                
        pm, rm, fm = evaluate(gp, "bilou", pp, "bilou", 1, 'special', per_dataset) #standard
        print("%s\t%s\t%f\t%f\t%f" % (event, case, pm, rm, fm))

        #gt, gl = read_bilou(gp)
        #pt, pl = read_bilou(pp)
        #pl = pad_unpredicted_seq(gl, pl)
        #ps = precision_score(gl, pl, mode='strict', scheme=BILOU)
        #rs = recall_score(gl, pl, mode='strict', scheme=BILOU)
        #fs = f1_score(gl, pl, mode='strict', scheme=BILOU)
        #print("%f\t%f" % (fm, fs))



california_wildfires_2018	random	0.722454	0.769505	0.728205
canada_wildfires_2016	random	0.426923	0.464744	0.435110
cyclone_idai_2019	random	0.255769	0.233755	0.240403
ecuador_earthquake_2016	random	0.265185	0.232593	0.242222
greece_wildfires_2018	random	0.832897	0.800611	0.792122
hurricane_dorian_2019	random	0.584936	0.616818	0.588684
hurricane_florence_2018	random	0.434986	0.547454	0.465517
hurricane_harvey_2017	random	0.362677	0.471042	0.396764
hurricane_irma_2017	random	0.338114	0.445736	0.368774
hurricane_maria_2017	random	0.447555	0.563707	0.478590
hurricane_matthew_2016	random	0.152196	0.138922	0.141260
italy_earthquake_aug_2016	random	0.728814	0.741525	0.729944
kaikoura_earthquake_2016	random	0.604603	0.596234	0.593863
kerala_floods_2018	random	0.688535	0.694551	0.664179
maryland_floods_2018	random	0.743590	0.803419	0.753755
midwestern_us_floods_2019	random	0.657193	0.764544	0.680145
pakistan_earthquake_2019	random	0.504415	0.336479	0.378797
puebla_mexico_earthquake_2017	random

In [118]:
from seqeval.metrics import precision_score, recall_score, f1_score
from seqeval.scheme import BILOU

path = "C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\data\\IDRISI\\en\\lmr\\"
events = ["earthquake", "flood", "hurricane", "wildfire"]
per_dataset = False

for typ in ["", "-LOC"]:
    if typ == "":
        print("BILOU")
    else:
        print("BILOU-LOC")
    for se in events:
        for de in events:
            pp = path + "baselines\\random\\BILOU" + typ + "-cleaned-transfer\\en-transfer-random-BILOU" + typ + "-cleaned-" + se + "-" + de
            gp = path + "annotations\\partitions\\random\\BILOU" + typ + "-cleaned-transfer\\" + se + "-" + de + "\\test.txt"            
            pm, rm, fm = evaluate(gp, "bilou", pp, "bilou", 1, 'special', per_dataset) #standard
            print("%s-%s\t%f\t%f\t%f" % (se, de, pm, rm, fm))

            #gt, gl = read_bilou(gp)
            #pt, pl = read_bilou(pp)
            #pl = pad_unpredicted_seq(gl, pl)
            #ps = precision_score(gl, pl, mode='strict', scheme=BILOU)
            #rs = recall_score(gl, pl, mode='strict', scheme=BILOU)
            #fs = f1_score(gl, pl, mode='strict', scheme=BILOU)
            #print("%f\t%f" % (fm, fs))



BILOU
earthquake-earthquake	0.926284	0.929355	0.922241
earthquake-flood	0.883650	0.872869	0.869655
earthquake-hurricane	0.810458	0.802069	0.799266
earthquake-wildfire	0.813278	0.811567	0.804271
flood-earthquake	0.865811	0.833149	0.840415
flood-flood	0.924923	0.916117	0.912842
flood-hurricane	0.819150	0.793853	0.798280
flood-wildfire	0.792228	0.781327	0.779395
hurricane-earthquake	0.893622	0.867842	0.873551
hurricane-flood	0.898406	0.892875	0.885803
hurricane-hurricane	0.886170	0.884070	0.879584
hurricane-wildfire	0.831327	0.827038	0.822240
wildfire-earthquake	0.795485	0.767212	0.773188
wildfire-flood	0.863512	0.839144	0.842041
wildfire-hurricane	0.817817	0.790099	0.796876
wildfire-wildfire	0.862551	0.868997	0.858338
BILOU-LOC
earthquake-earthquake	0.928550	0.929490	0.924081
earthquake-flood	0.904615	0.903217	0.894130
earthquake-hurricane	0.829152	0.827933	0.821763
earthquake-wildfire	0.858636	0.865130	0.853400
flood-earthquake	0.881672	0.864082	0.864948
flood-flood	0.924787	0.919268	0.

In [122]:
from seqeval.metrics import precision_score, recall_score, f1_score
from seqeval.scheme import BILOU

path = "C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\data\\IDRISI\\ar\\lmr\\"
events = ["flood", "bomb", "storm", "pandemic"]
per_dataset = False

for typ in ["", "-LOC"]:
    if typ == "":
        print("BILOU")
    else:
        print("BILOU-LOC")
    for se in events:
        for de in events:
            pp = path + "annotations\\7.baselines\\random\\BILOU" + typ + "-transfer\\idrisi-ar-MARBERT_NER-random-BILOU" + typ + "-transfer-" + se + "-" + de + ".txt"
            gp = path + "annotations\\6.partitions\\random\\BILOU" + typ + "-transfer\\" + se + "-" + de + "\\test.txt" 
            pm, rm, fm = evaluate(gp, "bilou", pp, "bilou", 1, 'special', per_dataset) #standard
            print("%s-%s\t%f\t%f\t%f" % (se, de, pm, rm, fm))

            #gt, gl = read_bilou(gp)
            #pt, pl = read_bilou(pp)
            #pl = pad_unpredicted_seq(gl, pl)
            #ps = precision_score(gl, pl, mode='strict', scheme=BILOU)
            #rs = recall_score(gl, pl, mode='strict', scheme=BILOU)
            #fs = f1_score(gl, pl, mode='strict', scheme=BILOU)
            #print("%f\t%f" % (fm, fs))



BILOU
flood-flood	0.926503	0.924633	0.921322
flood-bomb	0.818320	0.809294	0.800387
flood-storm	0.820175	0.814327	0.816667
flood-pandemic	0.885094	0.881213	0.877383
bomb-flood	0.270521	0.372907	0.291713
bomb-bomb	0.857576	0.852696	0.837485
bomb-storm	0.780702	0.723684	0.738012
bomb-pandemic	0.759748	0.741644	0.743899
storm-flood	0.284141	0.284141	0.284141
storm-bomb	0.090909	0.090909	0.090909
storm-storm	0.368421	0.368421	0.368421
storm-pandemic	0.566038	0.566038	0.566038
pandemic-flood	0.224018	0.296200	0.234173
pandemic-bomb	0.342975	0.263348	0.272746
pandemic-storm	0.622807	0.596491	0.592982
pandemic-pandemic	0.807008	0.793854	0.795867
BILOU-LOC
flood-flood	0.935463	0.931222	0.929758
flood-bomb	0.864325	0.866490	0.855864
flood-storm	0.838596	0.839181	0.833974
flood-pandemic	0.873145	0.863351	0.862091
bomb-flood	0.332054	0.475918	0.368873
bomb-bomb	0.851377	0.851397	0.840655
bomb-storm	0.807018	0.755848	0.767251
bomb-pandemic	0.848113	0.823109	0.826985
storm-flood	0.421615	0.587500	0.

In [173]:
path = "C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\data\\IDRISI\\ar\\lmr\\annotations\\"
gold_file_format = "bilou"
pred_file_format = "bio"
beta = 1
mode = "special"
per_dataset = False
events = ["beirut-explosion", "cairo-bombing", "covid-19", "dragon-storms",  "hafr-floods-19", 
          "jordan-floods", "kuwait-floods-18"]
models = ['bert-base-arabic-camelbert-ca-ner', 'bert-base-arabic-camelbert-da-ner', 
          'bert-base-arabic-camelbert-mix-ner', 'bert-base-arabic-camelbert-msa-ner']
usecases = ["random", "timebased"]
for usecase in usecases:
    for event in events:
        #print(event + "-" + usecase)
        true = path + "6.partitions\\" + usecase + "\\BILOU-LOC\\" + event + "\\test.txt"
        for model in models:
            pred = path + "7.baselines\\" + usecase + "\\camel\\" + event + "-" + model + "-test.txt"
            pm, rm, fm = evaluate(true, gold_file_format, pred, pred_file_format, beta, mode, per_dataset)
            run = event + "\t" + usecase + "\t" + model
            print("%s\t%f\t%f\t%f" % (run, pm, rm, fm))

beirut-explosion	random	bert-base-arabic-camelbert-ca-ner	0.457833	0.615712	0.503954
beirut-explosion	random	bert-base-arabic-camelbert-da-ner	0.434603	0.637451	0.485001
beirut-explosion	random	bert-base-arabic-camelbert-mix-ner	0.505602	0.637451	0.538581
beirut-explosion	random	bert-base-arabic-camelbert-msa-ner	0.497504	0.630205	0.531067
cairo-bombing	random	bert-base-arabic-camelbert-ca-ner	0.153846	0.057372	0.081777
cairo-bombing	random	bert-base-arabic-camelbert-da-ner	0.141026	0.058013	0.079945
cairo-bombing	random	bert-base-arabic-camelbert-mix-ner	0.375000	0.180449	0.237256
cairo-bombing	random	bert-base-arabic-camelbert-msa-ner	0.461538	0.250000	0.315781
covid-19	random	bert-base-arabic-camelbert-ca-ner	0.317381	0.424605	0.347074
covid-19	random	bert-base-arabic-camelbert-da-ner	0.280938	0.394627	0.311902
covid-19	random	bert-base-arabic-camelbert-mix-ner	0.193808	0.343194	0.237618
covid-19	random	bert-base-arabic-camelbert-msa-ner	0.201864	0.360647	0.248241
dragon-storms	rand

In [172]:
path = "C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\data\\IDRISI\\ar\\lmr\\annotations\\"
gold_file_format = "bio"
pred_file_format = "bio"
beta = 1
mode = "special"
per_dataset = False
##"beirut-explosion", "dragon-storms", "jordan-floods"

events = ["beirut-explosion", "cairo-bombing", "covid-19", "dragon-storms",  "hafr-floods-19", 
          "jordan-floods", "kuwait-floods-18"]
usecases = ["random", "timebased"]
for usecase in usecases:
    for event in events:
        run = event + "\t" + usecase + "\t"
        true = path + "6.partitions\\" + usecase + "\\BILOU-LOC\\" + event + "\\test.txt"
        pred = path + "7.baselines\\" + usecase + "\\farasa-ordered-penalize\\" + event + "-test.txt"
        pm, rm, fm = evaluate(true, gold_file_format, pred, pred_file_format, beta, mode, per_dataset)
        print("%s\t%f\t%f\t%f" % (run, pm, rm, fm))

beirut-explosion	random		0.492754	0.492754	0.492754
cairo-bombing	random		0.057692	0.057692	0.057692
covid-19	random		0.845283	0.845283	0.845283
dragon-storms	random		0.736842	0.736842	0.736842
hafr-floods-19	random		0.285714	0.285714	0.285714
jordan-floods	random		0.650485	0.650485	0.650485
kuwait-floods-18	random		0.687747	0.687747	0.687747
beirut-explosion	timebased		0.710145	0.710145	0.710145
cairo-bombing	timebased		0.086538	0.081731	0.083333
covid-19	timebased		0.800000	0.800000	0.800000
dragon-storms	timebased		0.701754	0.701754	0.701754
hafr-floods-19	timebased		0.275510	0.275510	0.275510
jordan-floods	timebased		0.640777	0.640777	0.640777
kuwait-floods-18	timebased		0.624506	0.624506	0.624506


In [175]:
path = "C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\data\\IDRISI\\ar\\lmr\\annotations\\"
gold_file_format = "bio"
pred_file_format = "bio"
beta = 1
mode = "special"
per_dataset = False
##"beirut-explosion", "dragon-storms", "jordan-floods"

events = ["beirut-explosion", "cairo-bombing", "covid-19", "dragon-storms",  "hafr-floods-19", 
          "jordan-floods", "kuwait-floods-18"]
usecases = ["random", "timebased"]
for usecase in usecases:
    for event in events:
        run = event + "\t" + usecase + "\t"
        true = path + "6.partitions\\" + usecase + "\\BILOU-LOC\\" + event + "\\test.txt"
        pred = path + "7.baselines\\" + usecase + "\\CRF-LOC\\" + event + "-test.txt"
        pm, rm, fm = evaluate(true, gold_file_format, pred, pred_file_format, beta, mode, per_dataset)
        print("%s\t%f\t%f\t%f" % (run, pm, rm, fm))

beirut-explosion	random		0.507246	0.507246	0.507246
cairo-bombing	random		0.439103	0.411859	0.396795
covid-19	random		0.871698	0.871698	0.871698
dragon-storms	random		0.631579	0.631579	0.631579
hafr-floods-19	random		0.267857	0.284014	0.271088
jordan-floods	random		0.539806	0.541262	0.540453
kuwait-floods-18	random		0.612648	0.610672	0.611331
beirut-explosion	timebased		0.623188	0.623188	0.623188
cairo-bombing	timebased		0.589744	0.602564	0.560256
covid-19	timebased		0.837736	0.837736	0.837736
dragon-storms	timebased		0.666667	0.666667	0.666667
hafr-floods-19	timebased		0.408163	0.406463	0.399660
jordan-floods	timebased		0.558252	0.572816	0.563107
kuwait-floods-18	timebased		0.527668	0.524374	0.525296


In [177]:
path = "C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\data\\IDRISI\\ar\\lmr\\annotations\\"
gold_file_format = "bio"
pred_file_format = "bio"
beta = 1
mode = "special"
per_dataset = False
##"beirut-explosion", "dragon-storms", "jordan-floods"

events = ["beirut-explosion", "cairo-bombing", "covid-19", "dragon-storms",  "hafr-floods-19", 
          "jordan-floods", "kuwait-floods-18"]
usecases = ["random", "timebased"]
for usecase in usecases:
    for event in events:
        run = event + "\t" + usecase + "\t"
        true = path + "6.partitions\\" + usecase + "\\BILOU-LOC\\" + event + "\\test.txt"
        pred = path + "7.baselines\\" + usecase + "\\MARBERT-LOC\\idrisi-ar-MARBERT_NER-" + usecase + "-BILOU-LOC-" + event 
        pm, rm, fm = evaluate(true, gold_file_format, pred, pred_file_format, beta, mode, per_dataset)
        print("%s\t%f\t%f\t%f" % (run, pm, rm, fm))

beirut-explosion	random		0.594203	0.594203	0.594203
cairo-bombing	random		0.947115	0.955128	0.950458
covid-19	random		0.943396	0.939623	0.940881
dragon-storms	random		0.853801	0.868421	0.857895
hafr-floods-19	random		0.849490	0.851190	0.845578
jordan-floods	random		0.920065	0.936084	0.925936
kuwait-floods-18	random		0.970356	0.972332	0.971218
beirut-explosion	timebased		0.431159	0.471014	0.441546
cairo-bombing	timebased		0.950321	0.964744	0.953022
covid-19	timebased		0.926415	0.923899	0.924780
dragon-storms	timebased		0.789474	0.807018	0.795322
hafr-floods-19	timebased		0.838435	0.851190	0.838435
jordan-floods	timebased		0.856958	0.910194	0.875081
kuwait-floods-18	timebased		0.930435	0.927866	0.927543


## Running LOC+TYPE evaluation

In [137]:
def filter_by_type(annot, typ):
    annot_type = []
    for i in range(len(annot)):
        a = []
        for j in range(len(annot[i])):
            if annot[i][j][1] == typ:
                a.append(annot[i][j])
                #print(annot[i][j])
                #print(a)
                #print()
        annot_type.append(a)

        #print(annot_type)
        #print()
        #print()
    return annot_type

In [39]:
ls = [[], [], [('puertorico', 'ISL'), ('norfolk', 'CITY'), ('virginia', 'STAT')], [('puertorico', 'CTRY')], 
      [('puertorico', 'ISL')], [], [('dominica92', 'ISL')], [('sanjuan', 'CITY'), ('us', 'CTRY')]]
print(len(ls))
fls = filter_by_type(ls, 'CTRY')
print(len(fls))
print(fls)

8
8
[[], [], [], [('puertorico', 'CTRY')], [], [], [], [('us', 'CTRY')]]


In [138]:
def filter_by_type2(annot, typ):
    annot_type = []
    for i in range(len(annot)):
        a = []
        if annot[i][1] == typ:
            a.append(annot[i])
            #print(annot[i])
            #print(a)
            #print()
        annot_type.append(a)

        #print(annot_type)
        #print()
        #print()
    return annot_type

In [144]:
def evaluate_by_type(gold_path, gold_file_format, pred_path, pred_file_format, beta, LOC, average, e_method):
    #DRIVER code

    # TODO 1. read the gold data 
    #         --> parameters: path, data format (BILOU, JSON)
    #         --> output gold: [[locs]], pred: [[locs]]
    gold = parse_bilou_by_type(gold_path) if gold_file_format == "bilou" else parse_json_by_type(gold_path, LOC)
    pred = parse_bilou_by_type(pred_path) if pred_file_format == "bilou" else parse_json_by_type(pred_path, LOC)
    
    #print(gold)
    #print(pred)
    
    f_gold = {item[1] for sublist in gold for item in sublist}
    f_pred = {item[1] for sublist in pred for item in sublist}
    types_to_eval = f_gold | f_pred
    #print(types_to_eval)
        
    ps = [] #every entry for one type
    rs = []
    fs = []
    
    #gold = [[item for sublist in gold[:10] for item in sublist]]
    #pred = [[item for sublist in pred[:10] for item in sublist]]
    #print(gold)
    #print(pred)
    
    if average == "macro":
        for typ in types_to_eval:
            gold_typ = filter_by_type(gold, typ)
            pred_typ = filter_by_type(pred, typ)
            #print(gold_typ)
            #print(pred_typ)
            ps.append(compute_precision(gold_typ, pred_typ, e_method))
            rs.append(compute_recall(gold_typ, pred_typ, e_method))
            fs.append(compute_fscore(beta, gold_typ, pred_typ, e_method))
            #print(ps)
            #print(rs)
            #print(fs)
            
        
        
        p = sum(ps)/len(ps)
        r = sum(rs)/len(rs)
        f = sum(fs)/len(fs)
        #print([(x, y) for x, y in zip(types_to_eval, fs)])

    else: #default average is micro
        #FIXME the special case is not handled here! this case is not implemented correctly!
        #some code in the last cell in this notebook
        p = 0
        r = 0
        f = 0

    return p, r, f


In [151]:
# test the BILOU and JSON parsing by type
path = "C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\data\\IDRISI\\en\\lmr\\"
LOC = False
average = 'macro'
beta = 1
e_method = 'special'#'standard'#


events = ["california_wildfires_2018", "canada_wildfires_2016", "cyclone_idai_2019", "ecuador_earthquake_2016", 
          "greece_wildfires_2018", "hurricane_dorian_2019", "hurricane_florence_2018", "hurricane_harvey_2017", 
          "hurricane_irma_2017", "hurricane_maria_2017", "hurricane_matthew_2016", "italy_earthquake_aug_2016", 
          "kaikoura_earthquake_2016", "kerala_floods_2018", "maryland_floods_2018", "midwestern_us_floods_2019", 
          "pakistan_earthquake_2019", "puebla_mexico_earthquake_2017", "srilanka_floods_2017"]

#events = ["california_wildfires_2018"]

for case in ["random", "timebased"]:
    for event in events:
        pp = path + "baselines\\" + case + "\\BERT\\en-target-" + case + "-BILOU-cleaned-" + event
        gp = path + "annotations\\partitions\\" + case + "\\JSON\\" + event + "\\test.txt"            
        pm, rm, fm = evaluate_by_type(gp, "json", pp, "bilou", beta, LOC, average, e_method)
        print("%s\t%s\t%f\t%f\t%f" % (event, case, pm, rm, fm))
        
        #gt, gl = read_bilou(gp.replace("JSON", "BILOU-cleaned"))
        #pt, pl = read_bilou(pp)
        
        #pl = pad_unpredicted_seq(gl, pl)
        #ps = precision_score(gl, pl, mode='strict', average=average, scheme=BILOU)
        #rs = recall_score(gl, pl, mode='strict', average=average, scheme=BILOU)
        #fs = f1_score(gl, pl, mode='strict', average=average, scheme=BILOU)
        #print("%f\t%f" % (fm, fs))

        

california_wildfires_2018	random	0.981399	0.982018	0.981410
canada_wildfires_2016	random	0.932244	0.933397	0.932167
cyclone_idai_2019	random	0.970645	0.971403	0.970282
ecuador_earthquake_2016	random	0.984074	0.984815	0.983827
greece_wildfires_2018	random	0.971827	0.972201	0.971129
hurricane_dorian_2019	random	0.964886	0.966489	0.965216
hurricane_florence_2018	random	0.957255	0.958159	0.957160
hurricane_harvey_2017	random	0.977661	0.977110	0.977018
hurricane_irma_2017	random	0.956425	0.957041	0.956539
hurricane_maria_2017	random	0.971879	0.971686	0.971544
hurricane_matthew_2016	random	0.970185	0.971307	0.969860
italy_earthquake_aug_2016	random	0.981462	0.981992	0.981638
kaikoura_earthquake_2016	random	0.980893	0.981729	0.980879
kerala_floods_2018	random	0.961895	0.963640	0.962251
maryland_floods_2018	random	0.932692	0.935096	0.933303
midwestern_us_floods_2019	random	0.980882	0.981885	0.980470
pakistan_earthquake_2019	random	0.946468	0.947808	0.945024
puebla_mexico_earthquake_2017	random

In [152]:
# test the BILOU and JSON parsing by type
path = "C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\data\\IDRISI\\en\\lmr\\"
LOC = False
average = 'macro'
beta = 1
e_method = 'special'#'standard'#


events = ["california_wildfires_2018", "canada_wildfires_2016", "cyclone_idai_2019", "ecuador_earthquake_2016", 
          "greece_wildfires_2018", "hurricane_dorian_2019", "hurricane_florence_2018", "hurricane_harvey_2017", 
          "hurricane_irma_2017", "hurricane_maria_2017", "hurricane_matthew_2016", "italy_earthquake_aug_2016", 
          "kaikoura_earthquake_2016", "kerala_floods_2018", "maryland_floods_2018", "midwestern_us_floods_2019", 
          "pakistan_earthquake_2019", "puebla_mexico_earthquake_2017", "srilanka_floods_2017"]

#events = ["california_wildfires_2018"]

for case in ["random", "timebased"]:
    for event in events:
        pp = path + "baselines\\" + case + "\\CRF-cleaned-ordered\\" + event + ".txt"
        gp = path + "annotations\\partitions\\" + case + "\\JSON\\" + event + "\\test.txt"            
        pm, rm, fm = evaluate_by_type(gp, "json", pp, "bilou", beta, LOC, average, e_method)
        print("%s\t%s\t%f\t%f\t%f" % (event, case, pm, rm, fm))
        
        #gt, gl = read_bilou(gp.replace("JSON", "BILOU-cleaned"))
        #pt, pl = read_bilou(pp)
        
        #pl = pad_unpredicted_seq(gl, pl)
        #ps = precision_score(gl, pl, mode='strict', average=average, scheme=BILOU)
        #rs = recall_score(gl, pl, mode='strict', average=average, scheme=BILOU)
        #fs = f1_score(gl, pl, mode='strict', average=average, scheme=BILOU)
        #print("%f\t%f" % (fm, fs))

        

california_wildfires_2018	random	0.982517	0.981685	0.981682
canada_wildfires_2016	random	0.937692	0.937628	0.937244
cyclone_idai_2019	random	0.975890	0.972660	0.973376
ecuador_earthquake_2016	random	0.979259	0.976852	0.977302
greece_wildfires_2018	random	0.970456	0.969708	0.969334
hurricane_dorian_2019	random	0.962571	0.961966	0.961677
hurricane_florence_2018	random	0.958712	0.958392	0.958144
hurricane_harvey_2017	random	0.970767	0.969664	0.970031
hurricane_irma_2017	random	0.966878	0.966643	0.966631
hurricane_maria_2017	random	0.971236	0.971171	0.971042
hurricane_matthew_2016	random	0.967191	0.965569	0.965719
italy_earthquake_aug_2016	random	0.977754	0.977225	0.977401
kaikoura_earthquake_2016	random	0.977057	0.976220	0.976192
kerala_floods_2018	random	0.960969	0.959815	0.960024
maryland_floods_2018	random	0.942308	0.944712	0.941850
midwestern_us_floods_2019	random	0.975854	0.974814	0.974228
pakistan_earthquake_2019	random	0.939924	0.937401	0.936487
puebla_mexico_earthquake_2017	random

In [178]:
path = "C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\data\\IDRISI\\ar\\lmr\\annotations\\"
gold_file_format = "bio"
pred_file_format = "bio"
beta = 1
mode = "special"
per_dataset = False
##"beirut-explosion", "dragon-storms", "jordan-floods"

events = ["beirut-explosion", "cairo-bombing", "covid-19", "dragon-storms",  "hafr-floods-19", 
          "jordan-floods", "kuwait-floods-18"]
usecases = ["random", "timebased"]
for usecase in usecases:
    for event in events:
        run = event + "\t" + usecase + "\t"
        true = path + "6.partitions\\" + usecase + "\\BILOU\\" + event + "\\test.txt"
        pred = path + "7.baselines\\" + usecase + "\\CRF\\" + event + "-test.txt"
        pm, rm, fm = evaluate(true, gold_file_format, pred, pred_file_format, beta, mode, per_dataset)
        print("%s\t%f\t%f\t%f" % (run, pm, rm, fm))

beirut-explosion	random		0.507246	0.507246	0.507246
cairo-bombing	random		0.455128	0.411859	0.405128
covid-19	random		0.894340	0.894340	0.894340
dragon-storms	random		0.701754	0.701754	0.701754
hafr-floods-19	random		0.282313	0.294218	0.284354
jordan-floods	random		0.566343	0.565534	0.565881
kuwait-floods-18	random		0.632411	0.630435	0.631094
beirut-explosion	timebased		0.681159	0.681159	0.681159
cairo-bombing	timebased		0.589744	0.602564	0.560256
covid-19	timebased		0.852830	0.852830	0.852830
dragon-storms	timebased		0.701754	0.701754	0.701754
hafr-floods-19	timebased		0.438776	0.426871	0.426871
jordan-floods	timebased		0.601942	0.601942	0.601942
kuwait-floods-18	timebased		0.540184	0.537549	0.538208


In [180]:
path = "C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\data\\IDRISI\\ar\\lmr\\annotations\\"
gold_file_format = "bio"
pred_file_format = "bio"
beta = 1
mode = "special"
per_dataset = False
##"beirut-explosion", "dragon-storms", "jordan-floods"

events = ["beirut-explosion", "cairo-bombing", "covid-19", "dragon-storms",  "hafr-floods-19", 
          "jordan-floods", "kuwait-floods-18"]
usecases = ["random", "timebased"]
for usecase in usecases:
    for event in events:
        run = event + "\t" + usecase + "\t"
        true = path + "6.partitions\\" + usecase + "\\BILOU\\" + event + "\\test.txt"
        pred = path + "7.baselines\\" + usecase + "\\MARBERT\\idrisi-ar-MARBERT_NER-" + usecase + "-BILOU-" + event 
        pm, rm, fm = evaluate(true, gold_file_format, pred, pred_file_format, beta, mode, per_dataset)
        print("%s\t%f\t%f\t%f" % (run, pm, rm, fm))

beirut-explosion	random		0.594203	0.594203	0.594203
cairo-bombing	random		0.929808	0.932692	0.929167
covid-19	random		0.909434	0.909434	0.909434
dragon-storms	random		0.754386	0.754386	0.754386
hafr-floods-19	random		0.751190	0.755102	0.748397
jordan-floods	random		0.825243	0.825243	0.825243
kuwait-floods-18	random		0.937606	0.939921	0.937921
beirut-explosion	timebased		0.782609	0.782609	0.782609
cairo-bombing	timebased		0.908654	0.935897	0.919139
covid-19	timebased		0.860377	0.860377	0.860377
dragon-storms	timebased		0.736842	0.736842	0.736842
hafr-floods-19	timebased		0.821429	0.829932	0.818027
jordan-floods	timebased		0.804854	0.805825	0.803668
kuwait-floods-18	timebased		0.887352	0.880764	0.882477


In [111]:
#reads the output of GazPNE and returns a dictionary of {tweet:{loc:[start_idx, end_idx]}}
def read_gazpne_results(path):
    tweets = {}
    next_ = 0
    sep = "**************************************************"
    first = True
    lines = open(path, encoding='utf-8').read().splitlines()
    for i in range(len(lines)):
        if i < next_:
            continue

        if i == len(lines):
            break
        
        if first:
            if lines[i] == sep:
                first = False
            next_ += 1
            continue
        
        tweet = lines[i]
        idxs = []
        locs = []

        idxs = json.loads(lines[i+1].replace(",)",")").replace("(", "[").replace(")", "]"))
        locs_strs = lines[i+2].replace(",)",")").split("), (")
        #print(locs_str)
        for locs_str in locs_strs:
            locs.append(locs_str.replace("(", "").replace(")", "").replace("[", "").replace("]", "").replace("'", "").split(','))
        
        tweets[tweet] = {}

        for i in range(len(idxs)):
            tweets[tweet]["".join(locs[i]).strip()] = idxs[i]
                
        next_ += 4

    return tweets

## EN datasets

In [11]:
path = "C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\data\\IDRISI\\en\\"


datasets = ["california_wildfires_2018", "canada_wildfires_2016", "cyclone_idai_2019", "ecuador_earthquake_2016", 
          "greece_wildfires_2018", "hurricane_dorian_2019", "hurricane_florence_2018", "hurricane_harvey_2017", 
          "hurricane_irma_2017", "hurricane_maria_2017", "hurricane_matthew_2016", "italy_earthquake_aug_2016", 
          "kaikoura_earthquake_2016", "kerala_floods_2018", "maryland_floods_2018", "midwestern_us_floods_2019", 
          "pakistan_earthquake_2019", "puebla_mexico_earthquake_2017", "srilanka_floods_2017"]


#datasets = ["california_wildfires_2018"]

for dataset in datasets:
    print(dataset)
    anno_path = path + "lmr\\annotations\\partitions\\batch\\BILOU\\" + dataset + "\\test.txt"
    pred_path = path + "lmr\\annotations\\GazPNE\\output\\batch-" + dataset + "-test-disabled-tokenizer.output"
    out_path = path + "lmr\\annotations\\GazPNE\\predictions\\batch\\" + dataset + ".txt"

    gazpne2bilou(anno_path, pred_path, out_path)
    #break


#The printed stuff is to be fixed manually!


california_wildfires_2018
length issue: check it manually! ***************************************
loc = southern california
s_idx = 157
start_idxs = [0, 2, 10, 13, 15, 23, 30, 35, 40, 47, 52, 58, 64, 74, 77, 81, 86, 91, 94, 98, 104, 108, 112, 118, 124, 125, 131, 134, 142, 146, 151, 154, 157, 166, 167, 178, 185, 190, 197, 199, 200, 205, 206]
e_idx = 176
end_idxs = [1, 9, 12, 14, 22, 29, 34, 39, 46, 51, 57, 63, 73, 76, 80, 85, 90, 93, 97, 103, 107, 111, 117, 123, 125, 130, 133, 141, 145, 150, 153, 156, 165, 167, 177, 184, 189, 197, 198, 200, 204, 206, 210]
*****
A quarter of a million people have been forced from their homes statewide as the Camp Fire in the north and two other major #fires -- Woolsey and Hill -- in Southern #California burned this weekend. #fire #news
[(0, 'O'), (1, 'O'), (2, 'O'), (3, 'O'), (4, 'O'), (5, 'O'), (6, 'O'), (7, 'O'), (8, 'O'), (9, 'O'), (10, 'O'), (11, 'O'), (12, 'O'), (13, 'O'), (14, 'O'), (15, 'O'), (16, 'O'), (17, 'O'), (18, 'O'), (19, 'O'), (20, 'O'),

s_idx = 93
start_idxs = [0, 10, 16, 20, 21, 28, 32, 40, 43, 47, 54, 55, 63, 70, 74, 76, 84, 90, 93, 100, 101, 109, 116, 121, 124, 128, 134, 136, 146, 153]
e_idx = 112
end_idxs = [9, 15, 19, 21, 27, 31, 39, 42, 46, 54, 55, 62, 69, 74, 75, 83, 89, 92, 100, 101, 108, 115, 120, 123, 127, 133, 135, 145, 153, 154]
length issue: check it manually! ***************************************
loc = jupiter inlet
s_idx = 76
start_idxs = [0, 10, 16, 20, 21, 28, 32, 40, 43, 47, 54, 55, 63, 70, 74, 76, 84, 90, 93, 100, 101, 109, 116, 121, 124, 128, 134, 136, 146, 153]
e_idx = 89
end_idxs = [9, 15, 19, 21, 27, 31, 39, 42, 46, 54, 55, 62, 69, 74, 75, 83, 89, 92, 100, 101, 108, 115, 120, 123, 127, 133, 135, 145, 153, 154]
*****
Hurricane watch for #Dorian now extends to the Flagler-Volusia county line. Jupiter Inlet to Volusia-Brevard county line is now under a hurricane warning.
[(0, 'O'), (1, 'O'), (2, 'O'), (3, 'O'), (4, 'O'), (5, 'O'), (6, 'O'), (7, 'O'), (8, 'O'), (9, 'B-LOC'), (10, 'I-LOC'), (11, 'I

end_idxs = [2, 7, 8, 14, 15, 26, 30, 38, 39, 45, 52, 55, 59, 67, 76, 77, 89]
*****
RT @GMA: WATCH: Rebuilding St. Maarten; Irmas damage to the tourism industry:  @WrightUps
[(0, 'O'), (1, 'O'), (2, 'O'), (3, 'O'), (4, 'O'), (5, 'B-LOC'), (6, 'I-LOC'), (7, 'L-LOC'), (8, 'O'), (9, 'O'), (10, 'O'), (11, 'O'), (12, 'O'), (13, 'O'), (14, 'O'), (15, 'O'), (16, 'O')]
[(0, 'RT'), (1, '@GMA'), (2, ':'), (3, 'WATCH'), (4, ':'), (5, 'Rebuilding'), (6, 'St.'), (7, 'Maarten'), (8, ';'), (9, 'Irmas'), (10, 'damage'), (11, 'to'), (12, 'the'), (13, 'tourism'), (14, 'industry'), (15, ':'), (16, '@WrightUps')]
[(0, 'O'), (1, 'O'), (2, 'O'), (3, 'O'), (4, 'O'), (5, 'O'), (6, 'B-ISL'), (7, 'L-ISL'), (8, 'O'), (9, 'O'), (10, 'O'), (11, 'O'), (12, 'O'), (13, 'O'), (14, 'O'), (15, 'O'), (16, 'O')]



length issue: check it manually! ***************************************
loc = sr 441
s_idx = 44
start_idxs = [0, 6, 8, 12, 15, 24, 32, 35, 41, 44, 50, 52, 56, 58, 62, 74, 81, 85, 93, 96, 103, 110, 118, 124, 126

loc = ellicott city
s_idx = 172
start_idxs = [0, 1, 4, 5, 11, 13, 26, 28, 36, 41, 51, 57, 60, 67, 73, 78, 84, 89, 92, 97, 99, 101, 103, 105, 109, 111, 114, 122, 134, 142, 149, 153, 158, 159, 164, 172, 175, 184, 188, 190, 198, 200, 202, 211, 221, 228, 233, 236]
e_idx = 185
end_idxs = [1, 4, 5, 11, 12, 25, 27, 35, 40, 50, 56, 59, 66, 72, 77, 83, 88, 91, 97, 98, 101, 102, 104, 109, 110, 113, 121, 133, 141, 148, 152, 158, 159, 163, 171, 174, 183, 188, 189, 198, 199, 201, 210, 220, 227, 232, 234, 246]
*****
»MGN-AFRICA® pin:263789F4 » Missing Natl Guardsman tried to rescue woman when swept away in flood: Hes a hero: As rushing floodwaters quickly filled the small-town streets of Ellicott City, Maryland, a National Guardsman jumped into »  @wizkidayo
[(0, 'O'), (1, 'O'), (2, 'O'), (3, 'U-LOC'), (4, 'O'), (5, 'O'), (6, 'O'), (7, 'O'), (8, 'O'), (9, 'O'), (10, 'O'), (11, 'O'), (12, 'O'), (13, 'O'), (14, 'O'), (15, 'O'), (16, 'O'), (17, 'O'), (18, 'O'), (19, 'O'), (20, 'O'), (21, 'O'), (22, 'O'

# UNDP tutorial

In [14]:
path = "C:\\Users\\rs081123\\OneDrive - Qatar University\\research\\workspace\\_IDRISI\\"
per_dataset = False

for s in ["ALL4UNDP-QCRI-tutorial-TB", "ALL4UNDP-QCRI-tutorial-TL", "ALL4UNDP-QCRI-tutorial-TL-ch", 
          "ALL4UNDP-QCRI-tutorial-TL-ho", "ALL4UNDP-QCRI-tutorial-TL-lo", "ALL4UNDP-QCRI-tutorial-TL-ny", 
          "ALL4UNDP-QCRI-tutorial-TL-nz",
          "ALL4UNDP-QCRI-tutorial-partitioned-TB", "ALL4UNDP-QCRI-tutorial-partitioned-TL", 
          "ALL4UNDP-QCRI-tutorial-partitioned-TL-ch", "ALL4UNDP-QCRI-tutorial-partitioned-TL-ho",
          "ALL4UNDP-QCRI-tutorial-partitioned-TL-lo", "ALL4UNDP-QCRI-tutorial-partitioned-TL-ny",
          "ALL4UNDP-QCRI-tutorial-partitioned-TL-nz", "ALL4UNDP-QCRI-tutorial-partitioned-TL-hd",
          "DORIAN4UNDP-QCRI-tutorial-TL", "DORIAN4UNDP-QCRI-tutorial-partitioned-TL"]:
    pp = path + s + "\\test_predictions.txt" 
    gp = path + s + "\\test.txt"            
    pm, rm, fm = evaluate(gp, "bilou", pp, "bilou", 1, 'special', per_dataset) #standard
    print("%s\t%f\t%f\t%f" % (s, pm, rm, fm))



ALL4UNDP-QCRI-tutorial-TB	0.568287	0.568287	0.568287
ALL4UNDP-QCRI-tutorial-TL	1.000000	1.000000	1.000000
ALL4UNDP-QCRI-tutorial-TL-ch	0.200000	0.200000	0.200000
ALL4UNDP-QCRI-tutorial-TL-ho	0.326667	0.326667	0.326667
ALL4UNDP-QCRI-tutorial-TL-lo	0.360000	0.360000	0.360000
ALL4UNDP-QCRI-tutorial-TL-ny	0.472222	0.472222	0.472222
ALL4UNDP-QCRI-tutorial-TL-nz	0.669173	0.669173	0.669173
ALL4UNDP-QCRI-tutorial-partitioned-TB	0.907593	0.905891	0.900776
ALL4UNDP-QCRI-tutorial-partitioned-TL	0.907593	0.905891	0.900776
ALL4UNDP-QCRI-tutorial-partitioned-TL-ch	0.757222	0.665889	0.693921
ALL4UNDP-QCRI-tutorial-partitioned-TL-ho	0.787667	0.727000	0.746042
ALL4UNDP-QCRI-tutorial-partitioned-TL-lo	0.927222	0.880056	0.895024
ALL4UNDP-QCRI-tutorial-partitioned-TL-ny	0.818182	0.857113	0.826178
ALL4UNDP-QCRI-tutorial-partitioned-TL-nz	0.837093	0.841896	0.835983
ALL4UNDP-QCRI-tutorial-partitioned-TL-hd	0.574074	0.574074	0.574074
DORIAN4UNDP-QCRI-tutorial-TL	0.553241	0.553241	0.553241
DORIAN4UNDP-QCRI-tut

# Quality

In [21]:
#path ="C:\\Users\\rs081123\\OneDrive - Qatar University\\research\\workspace\\_IDRISI\\quality\\BILOU-LOC-cleaned\\"
path ="C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\workspace\\_IDRISI\\quality\\BILOU-LOC-cleaned\\"
per_dataset = False

for s in ["HRC-IDRISI2IDRISI", "HRC-IDRISI2IDRISI-1", "HRC-IDRISI2IDRISI-2", "HRC-IDRISI2IDRISI-3", "HRC-IDRISI2NY", 
          "HRC-IDRISI2NY-1", "HRC-IDRISI2NY-2", "HRC-IDRISI2NY-3", "HRC-NY2IDRISI", "HRC-NY2NY"]:
    pp = path + s + "\\test_predictions.txt" 
    gp = path + s + "\\test.txt"            
    pm, rm, fm = evaluate(gp, "bilou", pp, "bilou", 1, 'special', per_dataset) #standard
    print("%s\t%f\t%f\t%f" % (s, pm, rm, fm))
    TPs, FPs, FNs = count_matrix_per_example(gp, pp)
    #print(TPs)
    #print(FPs)
    #print(FNs)
    print("%s\tTPs=%f\tFPs=%f\tFNs=%f" % ("Matrix", sum(TPs), sum(FPs), sum(FNs)))
    #break


HRC-IDRISI2IDRISI	0.857555	0.877468	0.863379
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [17]:
#path ="C:\\Users\\rs081123\\OneDrive - Qatar University\\research\\workspace\\_IDRISI\\quality\\BILOU-LOC-cleaned\\"
path ="C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\workspace\\_IDRISI\\quality\\BILOU-LOC-cleaned\\"
per_dataset = False

for s in ["EQK-IDRISI2IDRISI", "EQK-IDRISI2IDRISI-1", "EQK-IDRISI2IDRISI-2", "EQK-IDRISI2OLIMAT", 
          "EQK-IDRISI2OLIMAT-1", "EQK-IDRISI2OLIMAT-2", "EQK-OLIMAT2IDRISI", "EQK-OLIMAT2OLIMAT"]:
    pp = path + s + "\\test_predictions.txt" 
    gp = path + s + "\\test.txt"            
    pm, rm, fm = evaluate(gp, "bilou", pp, "bilou", 1, 'special', per_dataset) #standard
    print("%s\t%f\t%f\t%f" % (s, pm, rm, fm))

EQK-IDRISI2IDRISI	0.850833	0.857372	0.850653
EQK-IDRISI2IDRISI-1	0.779808	0.783333	0.778046
EQK-IDRISI2IDRISI-2	0.911538	0.919872	0.912015
EQK-IDRISI2OLIMAT	0.823308	0.831245	0.823320
EQK-IDRISI2OLIMAT-1	0.834586	0.839390	0.833166
EQK-IDRISI2OLIMAT-2	0.781955	0.798663	0.785225
EQK-OLIMAT2IDRISI	0.105769	0.100000	0.101538
EQK-OLIMAT2OLIMAT	0.919799	0.921679	0.917771


In [16]:
#path ="C:\\Users\\rs081123\\OneDrive - Qatar University\\research\\workspace\\_IDRISI\\quality\\BILOU-LOC-cleaned\\"
path ="C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\workspace\\_IDRISI\\quality\\BILOU-LOC-cleaned\\"
per_dataset = False

for s in ["FLD-IDRISI2IDRISI", "FLD-IDRISI2OLIMAT", "FLD-OLIMAT2IDRISI", "FLD-OLIMAT2OLIMAT"]:
    pp = path + s + "\\test_predictions.txt" 
    gp = path + s + "\\test.txt"            
    pm, rm, fm = evaluate(gp, "bilou", pp, "bilou", 1, 'special', per_dataset) #standard
    print("%s\t%f\t%f\t%f" % (s, pm, rm, fm))
    
    

FLD-IDRISI2IDRISI	0.824038	0.822427	0.818785
FLD-IDRISI2OLIMAT	0.879722	0.836167	0.849802
FLD-OLIMAT2IDRISI	0.751827	0.772019	0.755806
FLD-OLIMAT2OLIMAT	0.839167	0.825611	0.822548


# Checking length of predictions and test

In [15]:
path = "C:\\Users\\QCRI-IMMRAN\\OneDrive - Qatar University\\research\\data\\IDRISI\\en\\"


datasets = ["california_wildfires_2018", "canada_wildfires_2016", "cyclone_idai_2019", "ecuador_earthquake_2016", 
          "greece_wildfires_2018", "hurricane_dorian_2019", "hurricane_florence_2018", "hurricane_harvey_2017", 
          "hurricane_irma_2017", "hurricane_maria_2017", "hurricane_matthew_2016", "italy_earthquake_aug_2016", 
          "kaikoura_earthquake_2016", "kerala_floods_2018", "maryland_floods_2018", "midwestern_us_floods_2019", 
          "pakistan_earthquake_2019", "puebla_mexico_earthquake_2017", "srilanka_floods_2017"]

for dataset in datasets:
    print(dataset)
    gold_path = path + "lmr\\annotations\\partitions\\realtime\\BILOU-LOC\\" + dataset + "\\test.txt"
    pred_path = path + "lmr\\annotations\\GazPNE\\predictions\\realtime\\" + dataset + ".txt"

    G_t, G_l = read_bilou(gold_path)
    P_t, P_l = read_bilou(pred_path)
    
    
    for i in range(len(G_t)):
        if len(G_t[i]) != len(P_t[i]):
            print("length issue: " +  str(i))


california_wildfires_2018
canada_wildfires_2016
cyclone_idai_2019
ecuador_earthquake_2016
greece_wildfires_2018
hurricane_dorian_2019
hurricane_florence_2018
hurricane_harvey_2017
hurricane_irma_2017
hurricane_maria_2017
hurricane_matthew_2016
italy_earthquake_aug_2016
kaikoura_earthquake_2016
kerala_floods_2018
maryland_floods_2018
midwestern_us_floods_2019
pakistan_earthquake_2019
puebla_mexico_earthquake_2017
srilanka_floods_2017


In [ ]:
        TPs = []
        FPs = []
        FNs = []
        s_counts = []
        beta2 = beta**2
        for typ in types_to_eval:
            gold_typ = filter_by_type(gold, typ)
            pred_typ = filter_by_type(pred, typ)
            s_counts.append(count_special(gold, pred))
            TP, FP, FN = count_matrix_per_example(gold_typ, pred_typ)
            TPs.extend(TP)
            FPs.extend(FP)
            FNs.extend(FN)
        
        #print(TPs)
        #print(FPs)
        #print(FNs)
        den = sum(TPs) + sum(FPs)
        p = (sum(TPs) / den) if den > 0 else 0
        p += sum(s_counts) / len(s_counts)
        den = sum(TPs) + sum(FNs)
        r = (sum(TPs) / den) if den > 0 else 0 
        r += sum(s_counts) / len(s_counts)
        f = ((1 + beta2) * p * r) / (beta2 * p + r)
  